In [ ]:
import sys
sys.path.insert(0, '../toys-data')
import conf

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import random

In [ ]:
project_id = '2024031800'

In [ ]:
url = f'{conf.get_base_url(project_id)}/shop/mall/prdt/prdt_list.php?pcate=033005&type=9&opt=sort_num&limit_e=90&col_sort=three&paging=1'
url = f'{conf.get_base_url(project_id)}/shop/mall/prdt/prdt_list.php?pcate=033005&type=9&opt=sort_num&limit_e=90&col_sort=three&paging=2'
session = requests.Session()

response = session.get(url, headers=conf.get_request_header(project_id))

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
html_obj_scripts = soup.find_all('script')

In [ ]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        if match.group(2) is not None:
            return ""
        else:
            return match.group(1) # captured quoted-string
    return regex.sub(_replacer, string)

In [ ]:
for html_obj_script in html_obj_scripts:
    try:
        text = str(html_obj_script)
        begin_text = r"dataLayer.push({"
        begin_index = text.find(begin_text)
        if begin_index == -1:
            continue
        end_text = r"});"
        end_index = text.find(end_text, begin_index)
        #
        js = (text[begin_index + len(begin_text) - 1: end_index + len(end_text) - 2])
        js = remove_comments(js)
        end_index =js.rfind(',')
        js = js[:end_index] + ']}}'
        js = json.loads(js)
        #
        for item in js['ecommerce']['items']:
            #print(item)
            url_item = f"{conf.get_base_url(project_id)}/shop/mall/prdt/prdt_view.php?pidx={item['item_id']}&pcate={item['item_category']}&cateChk=c&rdt=ok"
            response_item = session.get(url_item, headers=conf.get_request_header(project_id))
            time.sleep(random.randrange(1,7))
            soup_item = BeautifulSoup(response_item.content, 'html.parser')
            #
            select_item = soup_item.find('select')
            option_items = select_item.find_all('option')
            for option_item in option_items:
                if option_item.text.find('/40') != -1 and option_item.text.find('완판') == -1:
                    print(item['item_name'], option_item.text)
    except Exception as ex:
        print('[EXCEPTION]', ex)